In [13]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report


In [38]:
data = pd.read_csv("/Users/cjllop/Code/MIDS/MLearning/Final/Data/train.csv")
test = pd.read_csv("/Users/cjllop/Code/MIDS/MLearning/Final/Data/test.csv")


In [39]:
# Big Picture of Data
print "Train Data:"
print data.shape
print data.columns.values
print "Test Data:"
print test.shape
print test.columns.values

Train Data:
(878049, 9)
['Dates' 'Category' 'Descript' 'DayOfWeek' 'PdDistrict' 'Resolution'
 'Address' 'X' 'Y']
Test Data:
(884262, 7)
['Id' 'Dates' 'DayOfWeek' 'PdDistrict' 'Address' 'X' 'Y']


In [40]:
# Add a column counting days since the min date in the dataset
def add_date_diff(df):
    datetime_vector = pd.to_datetime(df['Dates'])
    date_vector = datetime_vector.dt.date
    date_diff_vector = (date_vector - date_vector.min()) / np.timedelta64(1, 'D')
    df['DateDiff'] = date_diff_vector

add_date_diff(data)
add_date_diff(test)
print data.DateDiff.describe()
print test.DateDiff.describe()

count    878049.000000
mean       2260.778323
std        1325.343365
min           0.000000
25%        1101.000000
50%        2252.000000
75%        3444.000000
max        4510.000000
Name: DateDiff, dtype: float64
count    884262.000000
mean       2259.105934
std        1327.543529
min           0.000000
25%        1088.000000
50%        2250.000000
75%        3444.000000
max        4512.000000
Name: DateDiff, dtype: float64


In [46]:
# Create random dev sample so we can see how that accuracy compares to our Kaggle results
np.random.seed(100)

rows = np.random.choice(data.index, size = len(data) / 10, replace = False)

dev = data.ix[rows]
train = data.drop(rows)

print train.shape
print dev.shape

(790245, 10)
(87804, 10)


In [45]:
# Convert to Numpy Format

# TODO 
test = train['DateDiff','X','Y']
#train_data = np.array(train['DateDiff','X','Y'])

KeyError: ('DateDiff', 'X', 'Y')

In [42]:
# Fit a basic KNN
KNNmodel = KNeighborsClassifier(n_neighbors=1)
KNNmodel.fit(train['DateDiff','X','Y'], train['Category'])
dev_predict = model.predict(dev)
print metrics.f1_score(dev['Category'], dev_predict, average='weighted')


KeyError: ('DateDiff', 'X', 'Y')

In [ ]:
    model = KNeighborsClassifier(n_neighbors=1)
    model.fit(mini_train_data, mini_train_labels)
    trian_predicted_labels = model.predict(dev_data)
metrics.f1_score(dev_labels, dev_predict, average='weighted')


    print; print "Part 1 - KNN:"
    ks = {'n_neighbors': range(1,2)}
    KNNGridSearch = GridSearchCV(KNeighborsClassifier(), ks, scoring='f1_weighted')
    KNNGridSearch.fit(train_csr_data, train_labels)
    # Report out on the accuracies    
    print "The best k value was %s with accuracy %.4f" % (KNNGridSearch.best_params_, KNNGridSearch.best_score_)
